In [ ]:
!pip install -Uqq fastai
!pip install -Uqq fastbook
!pip install -Uqq matplotlib
!pip install -Uqq scipy
!pip install -Uqq numpy

## Package Imports

In [ ]:
from fastbook import *
from fastai.vision.widgets  import *

import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import numpy as np

## Test audio download and Spectrogram Plot (Skip if needed)

In [ ]:
files = ['https://francisco-test.s3.us-east-2.amazonaws.com/Audio/SampleAI.wav']

dest = 'audio/sample.wav'
download_url(files[0], dest)

In [ ]:
sr, samples = wavfile.read(dest)
fqs, times, spectrogram = signal.spectrogram(samples, sr)

In [ ]:
plt.figure(figsize=(10, 4))
plt.pcolormesh(times, fqs, 10 * np.log10(spectrogram))
# plt.imshow(spectrogram)
plt.tight_layout()

plt.show()

## Batch file processing

To start from somewhere, I'll make this audio classification notebook a musical instrument classifier, but you can replace this with any audio category you want to work with.

Below is the source url of the files, an S3 bucket in this case, and in the `categories_dict` dictionary there's a key that represents a directory, and a value that represents the total number of files. The file names will follow the convention, s3_url/piano/piano001

In [ ]:
s3_bucket = 'https://francisco-test.s3.us-east-2.amazonaws.com/Audio/'
# dictionary with the categories and the number of wav files in each
categories_dict = {
    "drums": 40,
    "electricguitar": 16,
    "piano": 16,
    "voice": 29,
}

### Pad zeros helper
Function to pad an integer between 1-100 with zeros

In [ ]:
def pad_zeros(num):
    if not isinstance(num, int) or num < 1 or num > 100:
        raise ValueError("Input must be an integer between 1 and 100")

    padded_num = f"{num:03d}"
    return padded_num

## Download audio files

The files are stored in S3 in folders with the `categories_dict` keys as names, and the numeric value is the nu,ber of files on each category.

In [ ]:
import os.path

categories = 'drums', 'electricguitar', 'piano', 'voice'
path = Path('audio')
image_path = Path('images')

plt.figure(figsize=(10,4))

if not path.exists():
    path.mkdir()

if not image_path.exists():
    image_path.mkdir()

for category, num_files in categories_dict.items():
    dest = (path/category)
    image_dest = (image_path/category)
    print(dest, image_dest)
    
    dest.mkdir(exist_ok=True)
    image_dest.mkdir(exist_ok=True)
    
    # (audio url, audio file download destination, image file destination)[]
    files = [
        (
            f'{s3_bucket}{category}/{category}{pad_zeros(i)}.wav',
            f'{dest}/{pad_zeros(i)}.wav',
            f'{image_dest}/{pad_zeros(i)}.png'
        ) 
        for i in range(1,num_files+1)
    ]

    for ff in files:
        src = ff[0]
        destination = ff[1]
        image_destination = ff[2]
        
        # Download wav (Skip if already downloaded)
        check_audio_file = os.path.isfile(destination)
        if not check_audio_file:
            print(f'downloading {ff[0]} as {ff[1]}')
            download_url(src, destination)
        
        # Analyze wav file
        sr, samples = wavfile.read(destination)
        fqs, times, spectrogram = signal.spectrogram(samples, sr)
        plt.pcolormesh(times, fqs, 10 * np.log10(spectrogram))
        
        # Save image
        plt.savefig(image_destination)
        
print('Done')

## Transform audio files into spectrograms and save

## Images and verify

In [ ]:
fns = get_image_files(image_path)
fns

### Remove failed files

In [ ]:
failed = verify_images(fns)
failed

### Delete png file helper
In case you need to delete files to reprocess them

In [ ]:
def delete_png_files(directory):
    """
    Delete all files with a .png extension in the specified directory.

    Args:
        directory (str): Path to the directory where files will be deleted.
    """
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        # Check if the file is a regular file and ends with .png extension
        if os.path.isfile(file_path) and filename.lower().endswith('.png'):
            try:
                # Attempt to remove the file
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            except OSError as e:
                # Handle error (e.g., permission denied, file not found)
                print(f"Error deleting {file_path}: {e}")

# delete_png_files('images')

## Train data